In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
from itertools import islice

import requests
import json

In [3]:
path = 'C:/Users/82109/Desktop/deep daiv. 프로젝트'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 주어진 데이터 전처리

- `song_keyword_culst1.xlsx` : 노래마다 `기본 정보 + 요약된 노래 가사 + 그걸 텍스트 전처리한 결과 + af 클러스터링 결과` 다 통합한 상태
- `af_전체_장르.csv` : 전체 af DB, 장르 정보 포함
- `노래_전체_장르.csv` : 전체 노래 DB, 장르 정보 포함

<br>

- `song_data`에 클러스터링 csv merge해서 노래 DB에 af 기반 mood cluster 정보 추가
- 힙합 장르 제거

=> `song_clust.csv` : `기본 정보 + 요약된 노래 가사 + 그걸 텍스트 전처리한 결과 + af 클러스터링 결과 + mood label` + 힙합 제거

In [12]:
total_af = pd.read_csv(path + '/힙합 제거 후 데이터 파일/af_전체_장르.csv',index_col=0)
print(total_af.shape)
total_af.head()

(25439, 13)


,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,0eFMbKCRw8KByXyWBw8WO7,0.529,0.469,1,-6.967,0,0.0358,0.0899,0.000015,0.8310,0.344,87.864,['k-pop']
1,7eJMfftS33KTjuF7lTsMCx,0.726,0.431,8,-8.765,0,0.1350,0.7310,0.000000,0.6960,0.348,144.026,"['sad lo-fi', 'sad rap']"
2,2CbGuO0LtVvbh3umN3mDwM,0.577,0.729,7,-7.113,1,0.2210,0.3940,0.000156,0.1440,0.692,180.065,['japanese teen pop']
3,1JRNrJKTQhfvxII8kkIcAQ,0.629,0.585,9,-7.969,1,0.0572,0.4300,0.000000,0.3570,0.296,81.973,['japanese teen pop']
4,57u1FaZlztApkcT5x5XNuD,0.596,0.707,1,-6.825,1,0.0490,0.1110,0.000000,0.0926,0.262,133.858,['japanese teen pop']


In [13]:
total_af['id'].duplicated().sum()

5012

In [14]:
drop_index = total_af[total_af['id'].duplicated()].index
total_af.drop(drop_index,inplace=True)
total_af = total_af.reset_index().iloc[:,1:]
print(total_af.shape)
total_af.head()

(20427, 13)


,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,0eFMbKCRw8KByXyWBw8WO7,0.529,0.469,1,-6.967,0,0.0358,0.0899,0.000015,0.8310,0.344,87.864,['k-pop']
1,7eJMfftS33KTjuF7lTsMCx,0.726,0.431,8,-8.765,0,0.1350,0.7310,0.000000,0.6960,0.348,144.026,"['sad lo-fi', 'sad rap']"
2,2CbGuO0LtVvbh3umN3mDwM,0.577,0.729,7,-7.113,1,0.2210,0.3940,0.000156,0.1440,0.692,180.065,['japanese teen pop']
3,1JRNrJKTQhfvxII8kkIcAQ,0.629,0.585,9,-7.969,1,0.0572,0.4300,0.000000,0.3570,0.296,81.973,['japanese teen pop']
4,57u1FaZlztApkcT5x5XNuD,0.596,0.707,1,-6.825,1,0.0490,0.1110,0.000000,0.0926,0.262,133.858,['japanese teen pop']


In [15]:
total_af_genre = total_af[['id','genre']]
total_af_genre

,id,genre
0,0eFMbKCRw8KByXyWBw8WO7,['k-pop']
1,7eJMfftS33KTjuF7lTsMCx,"['sad lo-fi', 'sad rap']"
2,2CbGuO0LtVvbh3umN3mDwM,['japanese teen pop']
3,1JRNrJKTQhfvxII8kkIcAQ,['japanese teen pop']
4,57u1FaZlztApkcT5x5XNuD,['japanese teen pop']
...,...,...
20422,1AJZ9OpSs6V1jzUqdkdWPf,"['art pop', 'pop']"
20423,3Q5c8dpE2aYMx97WnwRFGj,"['art pop', 'pop']"
20424,7aObrOB2jVZJ2vJ79lQGjU,"['art pop', 'pop']"
20425,7MlsUrIhUPvjoLZhjTLfFo,"['art pop', 'pop']"


In [16]:
total_af['id'].duplicated().sum()

0

**노래 DB에 af 기반 mood cluster 정보 추가**

cluster 번호마다, matching 된 mood로 labeling

In [17]:
one = pd.read_excel(path + "/노래 데이터 통합/song_keyword_culst1.xlsx", index_col=0)
two = pd.read_excel(path + "/노래 데이터 통합/song_keyword_culst2.xlsx", index_col=0)
thr = pd.read_excel(path + "/노래 데이터 통합/song_keyword_culst3.xlsx", index_col=0)
fou = pd.read_excel(path + "/노래 데이터 통합/song_keyword_culst4.xlsx", index_col=0)
fiv = pd.read_excel(path + "/노래 데이터 통합/song_keyword_culst5.xlsx", index_col=0)
six = pd.read_excel(path + "/노래 데이터 통합/song_keyword_culst6.xlsx", index_col=0)

one.head(1)

,id,url,Artist,PC1,PC2,PC3,clust,name,lyrics,lyrics_english,summary,key_word
0,7i5pytF8KaId51lIzatwdg,https://open.spotify.com/track/7i5pytF8KaId51l...,Fujii Kaze,0.755436,-0.31284,0.373133,0,Kiri Ga Naikara,キリがないから\nここで終わらすだけなの\nさもなけりゃ 永遠に 永遠に 彷徨うザマ\nあと...,Because there is no drill\nJust end here\nZama...,<pad> because there is no drill <unk> Because ...,drill unk drill unk drill end zama wanders for...


In [18]:
one.shape[0] + two.shape[0] + thr.shape[0] + fou.shape[0] + fiv.shape[0] + six.shape[0]

16766

In [22]:
song_data = pd.read_csv(path + '/힙합 제거 후 데이터 파일/노래_전체_장르.csv',index_col=0)
print(song_data.shape)
song_data.head()

(20166, 15)


,id,url,Artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,0eFMbKCRw8KByXyWBw8WO7,https://open.spotify.com/track/0eFMbKCRw8KByXy...,Jung Kook,0.529,0.469,1.0,-6.967,0.0,0.0358,0.0899,0.000015,0.8310,0.344,87.864,['k-pop']
1,7eJMfftS33KTjuF7lTsMCx,https://open.spotify.com/track/7eJMfftS33KTjuF...,Powfu,0.726,0.431,8.0,-8.765,0.0,0.1350,0.7310,0.000000,0.6960,0.348,144.026,"['sad lo-fi', 'sad rap']"
2,2CbGuO0LtVvbh3umN3mDwM,https://open.spotify.com/track/2CbGuO0LtVvbh3u...,Fujii Kaze,0.577,0.729,7.0,-7.113,1.0,0.2210,0.3940,0.000156,0.1440,0.692,180.065,['japanese teen pop']
3,1JRNrJKTQhfvxII8kkIcAQ,https://open.spotify.com/track/1JRNrJKTQhfvxII...,Fujii Kaze,0.629,0.585,9.0,-7.969,1.0,0.0572,0.4300,0.000000,0.3570,0.296,81.973,['japanese teen pop']
4,57u1FaZlztApkcT5x5XNuD,https://open.spotify.com/track/57u1FaZlztApkcT...,Fujii Kaze,0.596,0.707,1.0,-6.825,1.0,0.0490,0.1110,0.000000,0.0926,0.262,133.858,['japanese teen pop']


In [23]:
song_data.iloc[0]

id                                             0eFMbKCRw8KByXyWBw8WO7
url                 https://open.spotify.com/track/0eFMbKCRw8KByXy...
Artist                                                      Jung Kook
danceability                                                    0.529
energy                                                          0.469
key                                                               1.0
loudness                                                       -6.967
mode                                                              0.0
speechiness                                                    0.0358
acousticness                                                   0.0899
instrumentalness                                             0.000015
liveness                                                        0.831
valence                                                         0.344
tempo                                                          87.864
genre               

In [24]:
clust =pd.concat([one[['id','Artist','clust','name']],two[['id','Artist','clust','name']],thr[['id','Artist','clust','name']],fou[['id','Artist','clust','name']],fiv[['id','Artist','clust','name']],six[['id','Artist','clust','name']]])
clust

,id,Artist,clust,name
0,7i5pytF8KaId51lIzatwdg,Fujii Kaze,0,Kiri Ga Naikara
1,7r9v0CLTunOwlbvdOFFgZo,Fujii Kaze,0,Tsumi No Kaori
2,74Zrps4gc3qTF49oKpm6zF,Fujii Kaze,0,Cho Si Noccha Te
3,0o9zmvc5f3EFApU52PPIyW,Fujii Kaze,0,Shinunoga E-Wa
4,1KixkQVDUHggZMU9dUobgm,Ava Max,0,My Head & My Heart
...,...,...,...,...
1007,1KUQAGGba6Jh4rwAJnr0ww,Various Artists,5,I Am Still Running
1008,7ih5f8kBhnxLFxPbFSE4V0,Various Artists,5,No Time To Die
1009,1yDQNLe66vWAUQo2Mud7Mi,Various Artists,5,Astronomy
1010,5JL1HLtR9jrlQraBCT1DL9,Various Artists,5,We Don't Have To Take Our Clothes Off


In [25]:
song_clust = pd.merge(song_data, clust, how='inner',on='id')
print(song_clust.shape)
song_clust.head()

(16766, 18)


,id,url,Artist_x,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre,Artist_y,clust,name
0,0eFMbKCRw8KByXyWBw8WO7,https://open.spotify.com/track/0eFMbKCRw8KByXy...,Jung Kook,0.529,0.469,1.0,-6.967,0.0,0.0358,0.0899,0.000015,0.8310,0.344,87.864,['k-pop'],Jung Kook,4,Still With You
1,7eJMfftS33KTjuF7lTsMCx,https://open.spotify.com/track/7eJMfftS33KTjuF...,Powfu,0.726,0.431,8.0,-8.765,0.0,0.1350,0.7310,0.000000,0.6960,0.348,144.026,"['sad lo-fi', 'sad rap']",Powfu,5,death bed (coffee for your head)
2,2CbGuO0LtVvbh3umN3mDwM,https://open.spotify.com/track/2CbGuO0LtVvbh3u...,Fujii Kaze,0.577,0.729,7.0,-7.113,1.0,0.2210,0.3940,0.000156,0.1440,0.692,180.065,['japanese teen pop'],Fujii Kaze,3,Nan-Nan
3,1JRNrJKTQhfvxII8kkIcAQ,https://open.spotify.com/track/1JRNrJKTQhfvxII...,Fujii Kaze,0.629,0.585,9.0,-7.969,1.0,0.0572,0.4300,0.000000,0.3570,0.296,81.973,['japanese teen pop'],Fujii Kaze,3,Mo-Eh-Wa
4,57u1FaZlztApkcT5x5XNuD,https://open.spotify.com/track/57u1FaZlztApkcT...,Fujii Kaze,0.596,0.707,1.0,-6.825,1.0,0.0490,0.1110,0.000000,0.0926,0.262,133.858,['japanese teen pop'],Fujii Kaze,1,YASASHISA


In [26]:
def song_mood(x):
    if x == 0:
        return [9,7, 2,6]#['enthusiasm','love','fun','happiness'] #[9,7, 2,6]
    elif x == 1:
        return [8,12]#['hate','anger'] #[8,12]
    elif x == 2:
        return [1,0,6]#['empty','sadness','love'] #[1,0,6]
    elif x == 3:
        return [11]#['relief'] #[11]
    elif x == 4:
        return [4]#['worry'] #[4]
    else:
        return [10]#['boredom'] #[10]

In [27]:
song_clust['sentiment'] =song_clust['clust'].apply(song_mood)
print(song_clust.shape)
song_clust.head()

(16766, 19)


,id,url,Artist_x,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre,Artist_y,clust,name,sentiment
0,0eFMbKCRw8KByXyWBw8WO7,https://open.spotify.com/track/0eFMbKCRw8KByXy...,Jung Kook,0.529,0.469,1.0,-6.967,0.0,0.0358,0.0899,0.000015,0.8310,0.344,87.864,['k-pop'],Jung Kook,4,Still With You,[4]
1,7eJMfftS33KTjuF7lTsMCx,https://open.spotify.com/track/7eJMfftS33KTjuF...,Powfu,0.726,0.431,8.0,-8.765,0.0,0.1350,0.7310,0.000000,0.6960,0.348,144.026,"['sad lo-fi', 'sad rap']",Powfu,5,death bed (coffee for your head),[10]
2,2CbGuO0LtVvbh3umN3mDwM,https://open.spotify.com/track/2CbGuO0LtVvbh3u...,Fujii Kaze,0.577,0.729,7.0,-7.113,1.0,0.2210,0.3940,0.000156,0.1440,0.692,180.065,['japanese teen pop'],Fujii Kaze,3,Nan-Nan,[11]
3,1JRNrJKTQhfvxII8kkIcAQ,https://open.spotify.com/track/1JRNrJKTQhfvxII...,Fujii Kaze,0.629,0.585,9.0,-7.969,1.0,0.0572,0.4300,0.000000,0.3570,0.296,81.973,['japanese teen pop'],Fujii Kaze,3,Mo-Eh-Wa,[11]
4,57u1FaZlztApkcT5x5XNuD,https://open.spotify.com/track/57u1FaZlztApkcT...,Fujii Kaze,0.596,0.707,1.0,-6.825,1.0,0.0490,0.1110,0.000000,0.0926,0.262,133.858,['japanese teen pop'],Fujii Kaze,1,YASASHISA,"[8, 12]"


**힙합 장르 제거**

In [28]:
type(song_clust['genre'])

pandas.core.series.Series

In [29]:
## list 형태의 장르 열을 string 형태로 변환
import ast

def to_list(x):
    return ast.literal_eval(x)

song_clust['genre'] = song_clust['genre'].apply(to_list)

In [30]:
## 힘합 장르를 포함하는 경우를 찾아내는 함수
def hiphop(x):
    exists = 0
    for i in x:
        if 'hip hop' in i.replace('-',' '):
            exists = 1
    return exists

In [31]:
song_clust['genre'].value_counts()

[reggaeton, reggaeton colombiano, trap latino, urbano latino]    913
[pop]                                                            859
[canadian contemporary r&b, canadian pop, pop]                   643
[]                                                               505
[hip hop, rap, slap house]                                       224
                                                                ... 
[canadian garage rock, toronto indie]                              1
[cubaton]                                                          1
[pop, uk dance, uk funky]                                          1
[tamil pop, tollywood]                                             1
[desi pop, hindi indie]                                            1
Name: genre, Length: 742, dtype: int64

In [32]:
song_clust['exists'] = song_clust['genre'].apply(hiphop)
song_clust['exists'].sum()

2790

In [33]:
drop_index = song_clust[song_clust['exists'] == 1].index
song_clust.drop(drop_index,inplace=True)
song_clust = song_clust.drop(['exists'],axis=1)

In [38]:
song_clust = song_clust.reset_index().iloc[:,1:]
print(song_clust.shape)
song_clust.head()

(13976, 19)


,id,url,Artist_x,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre,Artist_y,clust,name,sentiment
0,0eFMbKCRw8KByXyWBw8WO7,https://open.spotify.com/track/0eFMbKCRw8KByXy...,Jung Kook,0.529,0.469,1.0,-6.967,0.0,0.0358,0.0899,0.000015,0.8310,0.344,87.864,[k-pop],Jung Kook,4,Still With You,[4]
1,7eJMfftS33KTjuF7lTsMCx,https://open.spotify.com/track/7eJMfftS33KTjuF...,Powfu,0.726,0.431,8.0,-8.765,0.0,0.1350,0.7310,0.000000,0.6960,0.348,144.026,"[sad lo-fi, sad rap]",Powfu,5,death bed (coffee for your head),[10]
2,2CbGuO0LtVvbh3umN3mDwM,https://open.spotify.com/track/2CbGuO0LtVvbh3u...,Fujii Kaze,0.577,0.729,7.0,-7.113,1.0,0.2210,0.3940,0.000156,0.1440,0.692,180.065,[japanese teen pop],Fujii Kaze,3,Nan-Nan,[11]
3,1JRNrJKTQhfvxII8kkIcAQ,https://open.spotify.com/track/1JRNrJKTQhfvxII...,Fujii Kaze,0.629,0.585,9.0,-7.969,1.0,0.0572,0.4300,0.000000,0.3570,0.296,81.973,[japanese teen pop],Fujii Kaze,3,Mo-Eh-Wa,[11]
4,57u1FaZlztApkcT5x5XNuD,https://open.spotify.com/track/57u1FaZlztApkcT...,Fujii Kaze,0.596,0.707,1.0,-6.825,1.0,0.0490,0.1110,0.000000,0.0926,0.262,133.858,[japanese teen pop],Fujii Kaze,1,YASASHISA,"[8, 12]"


In [37]:
song_clust.to_excel('song_clust.xlsx')

## 학습 데이터 구축

- `new_data` : 전체 노래 DB에서, 각 노래가 해당하는 af 기반 sentiment 1-hot labeling
- `song_sentiment` : 전체 노래 DB에서, af 값 + sentiment 1-hot labeling 결과


<br>

- `traindata_af.pickle` 
- `new_data_2` : 책-OST DB에서, 각 노래가 해당하는 sentiment labeling
- `song_sentiment_2` : 책-OST DB에서, af 값 + sentiment 1-hot labeling 결과

In [ ]:
song_clust = pd.read_csv(path + '/audio feature 감정 추출 모델/song_clust.xlsx', index_col=0, encoding ='utf-8')
song_clust['sentiment'].values

In [39]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(song_clust['sentiment'].values)
print(labels)
mlb.classes_

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 ...
 [0 0 0 ... 1 0 0]
 [1 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]]


array([ 0,  1,  2,  4,  6,  7,  8,  9, 10, 11, 12])

In [40]:
columns = ['empty','sadness','enthusiasm','worry','love','fun','hate','happiness','boredom','relief','anger']

new_data = pd.DataFrame(columns = columns, data=labels)
print(new_data.shape)
new_data.head()

(13976, 11)


,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,1,0,0,0,1


In [41]:
audio_feature = song_clust.iloc[:,3:-5]
song_sentiment = pd.concat([audio_feature, new_data],axis=1)
print(song_sentiment.shape)
song_sentiment.head()

(13976, 22)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
0,0.529,0.469,1.0,-6.967,0.0,0.0358,0.0899,0.000015,0.8310,0.344,...,0,0,1,0,0,0,0,0,0,0
1,0.726,0.431,8.0,-8.765,0.0,0.1350,0.7310,0.000000,0.6960,0.348,...,0,0,0,0,0,0,0,1,0,0
2,0.577,0.729,7.0,-7.113,1.0,0.2210,0.3940,0.000156,0.1440,0.692,...,0,0,0,0,0,0,0,0,1,0
3,0.629,0.585,9.0,-7.969,1.0,0.0572,0.4300,0.000000,0.3570,0.296,...,0,0,0,0,0,0,0,0,1,0
4,0.596,0.707,1.0,-6.825,1.0,0.0490,0.1110,0.000000,0.0926,0.262,...,0,0,0,0,0,1,0,0,0,1


In [42]:
# 책 OST에서 추출된 af
import pickle

with open(path + '/audio feature 감정 추출 모델/traindata_af.pickle', 'rb') as f:
    book_af = pickle.load(f)
print(book_af.shape)
book_af.head(3)

(115, 17)


,책제목,artist,song_name,id,spotify_artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,mood
0,두고 온 여름,권현정(소공녀OST),난 너만 있으면 되는데,4MzcQtRM0xlUhs5Y0Quy1R,Kwon Hyun Jung,0.525,0.00413,5.0,-26.576,1.0,0.1190,0.995,0.916,0.0770,0.196,66.221,"[empty, sadness, worry, boredom]"
1,두고 온 여름,Hideyuki Hashimoto,hoho,48Jivh6jAj87eK6lj3FtUo,Hideyuki Hashimoto,0.445,0.03670,1.0,-31.319,1.0,0.0433,0.991,0.944,0.0896,0.130,75.396,"[empty, sadness, worry, boredom]"
2,두고 온 여름,Hideyuki Hashimoto,Setsu,1GKz0TpC7HktxQTirXpyZi,Hideyuki Hashimoto,0.493,0.04960,0.0,-26.935,1.0,0.0469,0.989,0.908,0.1180,0.212,57.756,"[empty, sadness, worry, boredom]"


In [43]:
book_af['mood'].value_counts()

[happiness, love, fun]                13
[empty, sadness]                      12
[worry, sadness]                      10
[happiness, love]                     10
[sadness, love, happiness, relief]     8
[fun, happiness, relief]               7
[sadness, empty, worry]                7
[sadness, empty]                       6
[relief, happiness, love]              6
[worry, empty]                         6
[boredom, empty, sadness]              5
[enthusiasm, happiness]                5
[relief]                               4
[worry, relief]                        3
[anger]                                3
[empty, sadness, worry, boredom]       3
[happiness]                            2
[anger, worry]                         1
[enthusiasm, worry]                    1
[fun, love, happiness]                 1
[fun, enthusiasm]                      1
[anger, sadness, worry]                1
Name: mood, dtype: int64

In [44]:
word_to_index = {'empty':0,'sadness':1,'enthusiasm':2,'worry':3,'love':4,'fun':5,'hate':6,'happiness':7,'boredom':8,'relief':9,'anger':10}
word_to_index

{'empty': 0,
 'sadness': 1,
 'enthusiasm': 2,
 'worry': 3,
 'love': 4,
 'fun': 5,
 'hate': 6,
 'happiness': 7,
 'boredom': 8,
 'relief': 9,
 'anger': 10}

In [45]:
def one_hot_encoding(word, word_to_index):
    one_hot_vector = [0]*(len(word_to_index))
    for i in word:
        index = word_to_index[i]
        one_hot_vector[index] = 1
    return one_hot_vector

In [46]:
label_list =[]
for i in book_af['mood']:
    label_list.append(one_hot_encoding(i,word_to_index))
    
label_list[:5]

[[1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
 [1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
 [1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
 [0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0],
 [0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0]]

In [47]:
new_data_2 = pd.DataFrame(columns=columns, data = label_list)
print(new_data_2.shape)
new_data_2.head()

(115, 11)


,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
0,1,1,0,1,0,0,0,0,1,0,0
1,1,1,0,1,0,0,0,0,1,0,0
2,1,1,0,1,0,0,0,0,1,0,0
3,0,1,0,0,1,0,0,1,0,1,0
4,0,1,0,0,1,0,0,1,0,1,0


In [48]:
audio_feature_2 = book_af.iloc[:,5:-1]
song_sentiment_2 = pd.concat([audio_feature_2,new_data_2],axis=1)
print(song_sentiment_2.shape)
song_sentiment_2.head()

(115, 22)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
0,0.525,0.00413,5.0,-26.576,1.0,0.1190,0.995,0.916,0.0770,0.196,...,1,0,1,0,0,0,0,1,0,0
1,0.445,0.03670,1.0,-31.319,1.0,0.0433,0.991,0.944,0.0896,0.130,...,1,0,1,0,0,0,0,1,0,0
2,0.493,0.04960,0.0,-26.935,1.0,0.0469,0.989,0.908,0.1180,0.212,...,1,0,1,0,0,0,0,1,0,0
3,0.514,0.05690,0.0,-30.029,0.0,0.0352,0.993,0.925,0.1020,0.090,...,1,0,0,1,0,0,1,0,1,0
4,0.241,0.05650,6.0,-23.980,0.0,0.0408,0.992,0.882,0.0984,0.104,...,1,0,0,1,0,0,1,0,1,0


# Training

- index = 114까지 book_ost 데이터
- book_ost 데이터는 모두 학습데이터로 포함

In [49]:
song_sentiment.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'empty', 'sadness', 'enthusiasm', 'worry', 'love', 'fun', 'hate',
       'happiness', 'boredom', 'relief', 'anger'],
      dtype='object')

In [50]:
train_data = pd.concat([song_sentiment_2, song_sentiment],axis=0,ignore_index=True)
print(train_data.shape)
train_data.head()

(14091, 22)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
0,0.525,0.00413,5.0,-26.576,1.0,0.1190,0.995,0.916,0.0770,0.196,...,1,0,1,0,0,0,0,1,0,0
1,0.445,0.03670,1.0,-31.319,1.0,0.0433,0.991,0.944,0.0896,0.130,...,1,0,1,0,0,0,0,1,0,0
2,0.493,0.04960,0.0,-26.935,1.0,0.0469,0.989,0.908,0.1180,0.212,...,1,0,1,0,0,0,0,1,0,0
3,0.514,0.05690,0.0,-30.029,0.0,0.0352,0.993,0.925,0.1020,0.090,...,1,0,0,1,0,0,1,0,1,0
4,0.241,0.05650,6.0,-23.980,0.0,0.0408,0.992,0.882,0.0984,0.104,...,1,0,0,1,0,0,1,0,1,0


In [51]:
X = train_data[['danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo']]
Y= train_data[['empty', 'sadness', 'enthusiasm', 'worry', 'love', 'fun', 'hate',
       'happiness', 'boredom', 'relief', 'anger']]

In [50]:
X

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.525,0.00413,5.0,-26.576,1.0,0.1190,0.995,0.916000,0.0770,0.196,66.221
1,0.445,0.03670,1.0,-31.319,1.0,0.0433,0.991,0.944000,0.0896,0.130,75.396
2,0.493,0.04960,0.0,-26.935,1.0,0.0469,0.989,0.908000,0.1180,0.212,57.756
3,0.514,0.05690,0.0,-30.029,0.0,0.0352,0.993,0.925000,0.1020,0.090,122.109
4,0.241,0.05650,6.0,-23.980,0.0,0.0408,0.992,0.882000,0.0984,0.104,67.128
...,...,...,...,...,...,...,...,...,...,...,...
14086,0.572,0.23100,0.0,-6.316,0.0,0.0334,0.953,0.000004,0.1110,0.332,96.732
14087,0.563,0.47300,0.0,-4.147,1.0,0.0361,0.177,0.000002,0.1080,0.342,69.926
14088,0.589,0.41200,11.0,-6.226,0.0,0.0363,0.720,0.000000,0.1530,0.703,74.954
14089,0.551,0.55000,10.0,-5.339,1.0,0.0286,0.777,0.000000,0.2020,0.342,101.190


In [51]:
Y

,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
0,1,1,0,1,0,0,0,0,1,0,0
1,1,1,0,1,0,0,0,0,1,0,0
2,1,1,0,1,0,0,0,0,1,0,0
3,0,1,0,0,1,0,0,1,0,1,0
4,0,1,0,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
14086,0,0,0,0,0,0,0,0,1,0,0
14087,0,0,0,0,0,0,1,0,0,0,1
14088,0,0,0,0,0,0,0,0,1,0,0
14089,1,1,0,0,1,0,0,0,0,0,0


**train-valid split**

- book_ost 데이터는 모두 학습 데이터로 사용하고
- 전체 노래 DB 데이터에서 split하도록

In [52]:
from sklearn.model_selection import train_test_split

X_train, X_test,Y_train,Y_test = train_test_split(X[115:], Y[115:], test_size=0.4, random_state=123)

X_train = train_data = pd.concat([X[:115], X_train],axis=0,ignore_index=True)
X_test = train_data = pd.concat([X[:115], X_test],axis=0,ignore_index=True)
Y_train = train_data = pd.concat([Y[:115], Y_train],axis=0,ignore_index=True)
Y_test = train_data = pd.concat([Y[:115], Y_test],axis=0,ignore_index=True)

In [53]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(8500, 11)
(8500, 11)
(5706, 11)
(5706, 11)


**모델 학습**

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [53]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [56]:
train_dataset = TensorDataset(torch.tensor(X_train.values, dtype=torch.float32).to(device),
                              torch.tensor(Y_train.values, dtype=torch.float32).to(device))
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)

In [75]:
test_dataset = TensorDataset(torch.tensor(X_test.values, dtype=torch.float32).to(device),
                              torch.tensor(Y_test.values, dtype=torch.float32).to(device))
test_dataloader = DataLoader(test_dataset, batch_size=256, shuffle=True)

In [8]:
class MyModel(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features

        self.model = nn.Sequential(
            nn.BatchNorm1d(in_features),
            nn.Linear(in_features, 800),
            nn.ReLU(),

            nn.BatchNorm1d(800),            # nn.Dropout(0.1),
            nn.Linear(800, 600),
            nn.ReLU(),

            nn.BatchNorm1d(600),            # nn.Dropout(0.1),
            nn.Linear(600, 400),
            nn.ReLU(),

            nn.BatchNorm1d(400),
            nn.Linear(400, out_features),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [58]:
X_train.shape[1], Y_train.shape[1]

(11, 11)

In [59]:
train_dataloader

In [96]:
model = MyModel(X_train.shape[1], Y_train.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 0.05, 10)
criterion = nn.BCELoss()
max_epoch = 30

In [97]:
for epoch in range(max_epoch):    
    model.train()
    for i, (x_batch, y_batch) in enumerate(train_dataloader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        preds = model(x_batch)

        optimizer.zero_grad()
        train_loss = criterion(preds, y_batch)
        train_loss.backward()
        optimizer.step()
        
        if i % 100 == 0:
            print(f'Epoch: {epoch}, train loss:{train_loss.item():12.5f}', end=", ")

    model.eval()
    with torch.no_grad():
        for i, (x_batch, y_batch) in enumerate(test_dataloader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            preds = model(x_batch)
            valid_loss = criterion(preds, y_batch)
            
            if i % 100 == 0:
                print(f'valid loss:{valid_loss.item():12.5f}')

Epoch: 0, train loss:     0.71823, valid loss:     0.15435
Epoch: 1, train loss:     0.15322, valid loss:     0.15085
Epoch: 2, train loss:     0.10886, valid loss:     0.13690
Epoch: 3, train loss:     0.06053, valid loss:     0.03711
Epoch: 4, train loss:     0.05269, valid loss:     0.04075
Epoch: 5, train loss:     0.04701, valid loss:     0.02165
Epoch: 6, train loss:     0.16840, valid loss:     0.03037
Epoch: 7, train loss:     0.04944, valid loss:     0.02625
Epoch: 8, train loss:     0.11793, valid loss:     0.03404
Epoch: 9, train loss:     0.04557, valid loss:     0.01339
Epoch: 10, train loss:     0.02279, valid loss:     0.02639
Epoch: 11, train loss:     0.06742, valid loss:     0.02109
Epoch: 12, train loss:     0.01888, valid loss:     0.01790
Epoch: 13, train loss:     0.02743, valid loss:     0.01896
Epoch: 14, train loss:     0.07457, valid loss:     0.03322
Epoch: 15, train loss:     0.01042, valid loss:     0.09437
Epoch: 16, train loss:     0.01176, valid loss:   

**validation set에 돌려보기**

- 모델 input으로 X_text 이용

In [98]:
model.eval()
with torch.no_grad():
    probs = model(torch.tensor(X_test.values, dtype=torch.float32).to(device))

In [99]:
probs = probs.cpu().numpy()
probs.shape

(5706, 11)

In [100]:
X_test.index

RangeIndex(start=0, stop=5706, step=1)

In [101]:
Y_train.columns

Index(['empty', 'sadness', 'enthusiasm', 'worry', 'love', 'fun', 'hate',
       'happiness', 'boredom', 'relief', 'anger'],
      dtype='object')

In [102]:
df = pd.DataFrame(probs, index=X_test.index, columns = Y_train.columns)
df

,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
0,0.275256,0.294684,0.067680,0.414073,0.329284,0.257404,0.002675,0.571722,0.070783,0.257363,0.004994
1,0.313797,0.360171,0.074607,0.525126,0.243545,0.263902,0.002605,0.526271,0.087957,0.375637,0.004942
2,0.294603,0.315632,0.071614,0.449824,0.322634,0.305093,0.002567,0.619334,0.139616,0.459778,0.005099
3,0.301516,0.378358,0.069185,0.546034,0.195756,0.218471,0.001646,0.429420,0.062893,0.302185,0.003287
4,0.529414,0.641548,0.067014,0.750942,0.120171,0.141197,0.001567,0.266672,0.041860,0.179600,0.004588
...,...,...,...,...,...,...,...,...,...,...,...
5701,0.000409,0.000350,0.000545,0.000156,0.000659,0.001244,0.000135,0.000966,0.999992,0.000239,0.000106
5702,0.001585,0.001515,1.000000,0.000031,1.000000,1.000000,0.000171,1.000000,0.000100,0.000125,0.000147
5703,0.000432,0.000521,0.000364,0.000247,0.000499,0.000899,0.000204,0.000504,0.999957,0.000374,0.000212
5704,0.010294,0.006966,0.000103,0.000157,0.003547,0.000100,0.000071,0.000121,0.000647,0.998914,0.000101


**모델 저장**

- CustomModel.pkl : 은서 언니, book_ost 데이터 추가 전 모델
- add_book_ost_CustomModel.pkl : 은서 언니 최종 모델
- add_book_ost_CustomModel (해원).pkl : 해원 다시 돌린 모델

In [110]:
import joblib

joblib.dump(model, path + '/audio feature 감정 추출 모델/add_book_ost_CustomModel (해원).pkl')

['C:/Users/82109/Desktop/deep daiv. 프로젝트/audio feature 감정 추출 모델/add_book_ost_CustomModel (해원).pkl']

## af 감정 확률 벡터 예측

- custom 모델로 돌린, 전체 노래 DB + book_ost의, af 기반 감정 확률 벡터 추출 결과

- `book_ost_CustomModel_sentiment.xlsx` : 은서 언니 최종 결과
- `book_ost_CustomModel_sentiment (해원).xlsx` : 해원 다시 돌린 모델

In [9]:
import joblib
model = joblib.load(path + '/audio feature 감정 추출 모델/해원 수정 후/add_book_ost_CustomModel (해원).pkl')
model

MyModel(
  (model): Sequential(
    (0): BatchNorm1d(11, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=11, out_features=800, bias=True)
    (2): ReLU()
    (3): BatchNorm1d(800, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Linear(in_features=800, out_features=600, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(600, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Linear(in_features=600, out_features=400, bias=True)
    (8): ReLU()
    (9): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Linear(in_features=400, out_features=11, bias=True)
    (11): Sigmoid()
  )
)

In [54]:
model.eval()
with torch.no_grad():
    probs = model(torch.tensor(X.values, dtype=torch.float32).to(device))

In [55]:
probs = probs.cpu().numpy()
df = pd.DataFrame(probs,index=X.index, columns = Y.columns)
df

,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
0,0.275256,0.294684,0.067680,0.414073,0.329284,0.257404,0.002675,0.571722,0.070783,0.257363,0.004994
1,0.313797,0.360171,0.074607,0.525126,0.243545,0.263902,0.002605,0.526271,0.087957,0.375637,0.004942
2,0.294603,0.315632,0.071614,0.449824,0.322634,0.305093,0.002567,0.619334,0.139616,0.459778,0.005099
3,0.301516,0.378358,0.069185,0.546034,0.195756,0.218471,0.001646,0.429420,0.062893,0.302185,0.003287
4,0.529414,0.641548,0.067014,0.750942,0.120171,0.141197,0.001567,0.266672,0.041860,0.179600,0.004588
...,...,...,...,...,...,...,...,...,...,...,...
14086,0.000795,0.000816,0.000563,0.000167,0.003699,0.003605,0.000040,0.004658,0.999864,0.001423,0.000036
14087,0.000279,0.000355,0.000142,0.000159,0.000135,0.000152,1.000000,0.000229,0.000105,0.000300,1.000000
14088,0.002833,0.003284,0.018591,0.000122,0.045248,0.019038,0.002511,0.013638,0.925294,0.000500,0.003770
14089,0.998075,0.998797,0.001807,0.005194,0.997169,0.001590,0.000293,0.003292,0.000092,0.002482,0.001205


In [56]:
df.shape

(14091, 11)

In [109]:
df.to_excel('book_ost_CustomModel_sentiment (해원).xlsx')

## 최종 데이터로 통합

- song_final_data : 노래 전체 DB 데이터프레임에 af 기반 확률 벡터 추출 결과 반영
- song_final_data_2 : book_ost 데이터프레임에 af 기반 확률 벡터 추출 결과 반영
- song_final_data_3 : 노래 데이터 최종본, af 기반 확률 벡터 추출 결과까지 반영

=> book_ost_song_final_data.xlsx : 전체 노래 DB + book_ost, af 기반 감정 확률 벡터 추출 결과 파일
=> af_sentiment.xlsx : 전체 노래 DB af 기반 감정 확률 벡터 추출 결과 파일

<br>

=> book_ost_song_final_data (해원).xlsx : 해원 수정 후 파일
=> af_sentiment (해원).xlsx : 해원 수정 후 파일

In [58]:
CustomModel_sentiment = pd.read_excel(path + '/audio feature 감정 추출 모델/해원 수정 후/book_ost_CustomModel_sentiment (해원).xlsx',index_col=0)

In [59]:
CustomModel_sentiment

,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
0,0.275256,0.294684,0.067680,0.414073,0.329284,0.257404,0.002675,0.571722,0.070783,0.257363,0.004994
1,0.313797,0.360171,0.074607,0.525126,0.243545,0.263902,0.002605,0.526271,0.087957,0.375637,0.004942
2,0.294603,0.315632,0.071614,0.449824,0.322634,0.305093,0.002567,0.619334,0.139616,0.459778,0.005099
3,0.301516,0.378358,0.069185,0.546034,0.195756,0.218471,0.001646,0.429420,0.062893,0.302185,0.003287
4,0.529414,0.641548,0.067014,0.750942,0.120171,0.141197,0.001567,0.266672,0.041860,0.179600,0.004588
...,...,...,...,...,...,...,...,...,...,...,...
14086,0.000795,0.000816,0.000563,0.000167,0.003699,0.003605,0.000040,0.004658,0.999864,0.001423,0.000036
14087,0.000279,0.000355,0.000142,0.000159,0.000135,0.000152,1.000000,0.000229,0.000105,0.000300,1.000000
14088,0.002833,0.003284,0.018591,0.000122,0.045248,0.019038,0.002511,0.013638,0.925294,0.000500,0.003770
14089,0.998075,0.998797,0.001807,0.005194,0.997169,0.001590,0.000293,0.003292,0.000092,0.002482,0.001205


- 노래 전체 DB 데이터프레임에 af 기반 확률 벡터 추출 결과 반영

In [61]:
song_clust['genre']

0                     [k-pop]
1        [sad lo-fi, sad rap]
2         [japanese teen pop]
3         [japanese teen pop]
4         [japanese teen pop]
                 ...         
13971          [art pop, pop]
13972          [art pop, pop]
13973          [art pop, pop]
13974          [art pop, pop]
13975          [art pop, pop]
Name: genre, Length: 13976, dtype: object

In [62]:
song_final_data = pd.concat([song_clust['id'], song_clust['name'], song_clust['Artist_x'], CustomModel_sentiment[115:].reset_index().iloc[:,1:], song_clust['genre']], axis=1)
song_final_data.columns = ['id', 'name', 'Artist', 'empty', 'sadness', 'enthusiasm', 'worry',
       'love', 'fun', 'hate', 'happiness', 'boredom', 'relief', 'anger', 'genre']

In [63]:
print(song_final_data.shape)
song_final_data.head()

(13976, 15)


,id,name,Artist,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger,genre
0,0eFMbKCRw8KByXyWBw8WO7,Still With You,Jung Kook,0.001346,0.001468,0.000573,0.999999,0.000172,0.000438,0.000084,0.000297,0.000171,0.000331,0.000293,[k-pop]
1,7eJMfftS33KTjuF7lTsMCx,death bed (coffee for your head),Powfu,0.000675,0.000728,0.000677,0.000237,0.001199,0.001109,0.000269,0.000683,0.999826,0.000080,0.000237,"[sad lo-fi, sad rap]"
2,2CbGuO0LtVvbh3umN3mDwM,Nan-Nan,Fujii Kaze,0.000721,0.000805,0.000329,0.000452,0.000202,0.000271,0.000053,0.000246,0.000452,0.999978,0.000072,[japanese teen pop]
3,1JRNrJKTQhfvxII8kkIcAQ,Mo-Eh-Wa,Fujii Kaze,0.002910,0.002584,0.000131,0.000329,0.002033,0.000132,0.000044,0.000142,0.000081,0.999633,0.000056,[japanese teen pop]
4,57u1FaZlztApkcT5x5XNuD,YASASHISA,Fujii Kaze,0.000334,0.000332,0.000241,0.000060,0.000185,0.000300,1.000000,0.000386,0.000421,0.000141,1.000000,[japanese teen pop]


In [73]:
song_final_data.to_excel(path + '/해원 수정 후/af_sentiment (해원).xlsx')

- book_ost 데이터프레임에 af 기반 확률 벡터 추출 결과 반영

In [64]:
book_data  = book_af.iloc[:,2:]
print(book_data.shape)
book_data.head()

(115, 15)


,song_name,id,spotify_artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,mood
0,난 너만 있으면 되는데,4MzcQtRM0xlUhs5Y0Quy1R,Kwon Hyun Jung,0.525,0.00413,5.0,-26.576,1.0,0.1190,0.995,0.916,0.0770,0.196,66.221,"[empty, sadness, worry, boredom]"
1,hoho,48Jivh6jAj87eK6lj3FtUo,Hideyuki Hashimoto,0.445,0.03670,1.0,-31.319,1.0,0.0433,0.991,0.944,0.0896,0.130,75.396,"[empty, sadness, worry, boredom]"
2,Setsu,1GKz0TpC7HktxQTirXpyZi,Hideyuki Hashimoto,0.493,0.04960,0.0,-26.935,1.0,0.0469,0.989,0.908,0.1180,0.212,57.756,"[empty, sadness, worry, boredom]"
3,If You Could See Me Now,53BeSVY0mpmucCIr0pDkPp,Jon Belle,0.514,0.05690,0.0,-30.029,0.0,0.0352,0.993,0.925,0.1020,0.090,122.109,"[sadness, love, happiness, relief]"
4,Arabesque (Debussy),6JU2EiLrNcmPuzRYC89x5N,Thomas Adès,0.241,0.05650,6.0,-23.980,0.0,0.0408,0.992,0.882,0.0984,0.104,67.128,"[sadness, love, happiness, relief]"


In [67]:
song_final_data_2 = pd.concat([book_data['id'], book_data['song_name'], book_data['spotify_artist'], CustomModel_sentiment[:115]], axis=1)
song_final_data_2.columns = ['id', 'name', 'Artist', 'empty', 'sadness', 'enthusiasm', 'worry',
       'love', 'fun', 'hate', 'happiness', 'boredom', 'relief', 'anger']

In [71]:
print(song_final_data_2.shape)
song_final_data_2.tail()

(115, 14)


,id,name,Artist,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
110,33B1XmfncHqnfkrYZIcHbD,Any Other Name,Thomas Newman,0.481616,0.601478,0.076465,0.757031,0.125863,0.150368,0.001649,0.287628,0.029409,0.131472,0.003902
111,0jONzUJvInw2qqyOEGDbzE,We Haven't Turned Around,Gomez,0.209573,0.185745,0.000185,0.009922,0.006041,0.000211,0.247419,0.000198,0.335595,0.120750,0.262177
112,5rzEX5tCbOSCzoH92dbExL,Tommib,Squarepusher,0.266815,0.310842,0.052117,0.367928,0.329706,0.182027,0.001563,0.494161,0.041272,0.361732,0.003681
113,1PPFlpZZXfGCDANeUaoZYe,Ikebana,Kevin Sherwood,0.421947,0.464277,0.032420,0.005233,0.003240,0.011254,0.000856,0.020065,0.158614,0.999145,0.002709
114,4TT9902mhftdlIDaz31Yz7,Alone in Kyoto,Aer,0.111815,0.085778,0.000339,0.002274,0.001374,0.000435,0.000478,0.000548,0.310000,0.996495,0.000442


- 두 df 합쳐서 최종 df 생성

In [137]:
song_final_data_3 = pd.concat([song_final_data_2, song_final_data],axis=0,ignore_index=True)
print(song_final_data_3.shape)
song_final_data_3

(14091, 14)


,id,name,Artist,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
0,4MzcQtRM0xlUhs5Y0Quy1R,난 너만 있으면 되는데,Kwon Hyun Jung,0.275256,0.294684,0.067680,0.414073,0.329284,0.257404,0.002675,0.571722,0.070783,0.257363,0.004994
1,48Jivh6jAj87eK6lj3FtUo,hoho,Hideyuki Hashimoto,0.313797,0.360171,0.074607,0.525126,0.243545,0.263902,0.002605,0.526271,0.087957,0.375637,0.004942
2,1GKz0TpC7HktxQTirXpyZi,Setsu,Hideyuki Hashimoto,0.294603,0.315632,0.071614,0.449824,0.322634,0.305093,0.002567,0.619334,0.139616,0.459778,0.005099
3,53BeSVY0mpmucCIr0pDkPp,If You Could See Me Now,Jon Belle,0.301516,0.378358,0.069185,0.546034,0.195756,0.218471,0.001646,0.429420,0.062893,0.302185,0.003287
4,6JU2EiLrNcmPuzRYC89x5N,Arabesque (Debussy),Thomas Adès,0.529414,0.641548,0.067014,0.750942,0.120171,0.141197,0.001567,0.266672,0.041860,0.179600,0.004588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14086,5JL1HLtR9jrlQraBCT1DL9,We Don't Have To Take Our Clothes Off,Various Artists,0.000795,0.000816,0.000563,0.000167,0.003699,0.003605,0.000040,0.004658,0.999864,0.001423,0.000036
14087,4RyYlpna1veWNpLvGDyqnE,Us,Various Artists,0.000279,0.000355,0.000142,0.000159,0.000135,0.000152,1.000000,0.000229,0.000105,0.000300,1.000000
14088,7aObrOB2jVZJ2vJ79lQGjU,L.O.V.E.,Various Artists,0.002833,0.003284,0.018591,0.000122,0.045248,0.019038,0.002511,0.013638,0.925294,0.000500,0.003770
14089,7MlsUrIhUPvjoLZhjTLfFo,Until I Found You - Em Beihold Version,Various Artists,0.998075,0.998797,0.001807,0.005194,0.997169,0.001590,0.000293,0.003292,0.000092,0.002482,0.001205


In [140]:
song_final_data_3.to_excel('book_ost_song_final_data (해원).xlsx')